In [2]:
qd.ApiConfig.api_key = '6cEMHULKqEC4GE_Uprht'
stock_data = qd.get('EOD/AAPL')
df=pd.DataFrame(stock_data)

In [3]:
df.head()

,Open,High,Low,Close,Volume,Dividend,Split,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume
Date,,,,,,,,,,,,
2013-09-03,493.10,500.60,487.35,488.580,11854600.0,0.0,1.0,15.619650,15.857223,15.437510,15.476472,331928800.0
2013-09-04,499.56,502.24,496.28,498.691,12322600.0,0.0,1.0,15.824280,15.909173,15.720381,15.796753,345032800.0
2013-09-05,500.25,500.68,493.64,495.270,8441700.0,0.0,1.0,15.846136,15.859757,15.636755,15.688388,236367600.0
2013-09-06,498.44,499.38,489.95,498.220,12840200.0,0.0,1.0,15.788802,15.818578,15.519869,15.781833,359525600.0
2013-09-09,505.00,507.92,503.48,506.170,12167400.0,0.0,1.0,15.996600,16.089095,15.948451,16.033661,340687200.0


In [4]:
df = df[['Adj_Open',  'Adj_High',  'Adj_Low',  'Adj_Close', 'Adj_Volume']]

In [5]:
df['HL_PCT'] = (df['Adj_High'] - df['Adj_Low'])/df['Adj_Close'] * 100

In [6]:
df['PCT_Change'] = (df['Adj_Close'] - df['Adj_Open'])/df['Adj_Open'] * 100

In [7]:
df.head()

,Adj_Open,Adj_High,Adj_Low,Adj_Close,Adj_Volume,HL_PCT,PCT_Change
Date,,,,,,,
2013-09-03,15.619650,15.857223,15.437510,15.476472,331928800.0,2.711941,-0.916650
2013-09-04,15.824280,15.909173,15.720381,15.796753,345032800.0,1.195129,-0.173953
2013-09-05,15.846136,15.859757,15.636755,15.688388,236367600.0,1.421447,-0.995502
2013-09-06,15.788802,15.818578,15.519869,15.781833,359525600.0,1.892738,-0.044138
2013-09-09,15.996600,16.089095,15.948451,16.033661,340687200.0,0.877176,0.231683


In [8]:
import math
from sklearn import preprocessing, svm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [9]:
forecast_col = 'Adj_Close'
df.fillna(value=-99999, inplace=True)
forecast_out = int(math.ceil(0.01 * len(df)))

In [10]:
df['label'] = df[forecast_col].shift(-forecast_out)

In [11]:
df.dropna(inplace=True)

In [12]:
X = np.array(df.drop(['label'], 1))
y = np.array(df['label'])
X = preprocessing.scale(X)
y = np.array(df['label'])

In [13]:
x_train, x_test, y_train, y_test = train_test_split(X,y, train_size=0.5)

In [14]:
clf = svm.SVR()
clf.fit(x_train, y_train)
confidence = clf.score(x_test, y_test)
print(confidence)

0.9524870560455925


In [15]:
clf = LinearRegression()
clf = LinearRegression(n_jobs=-1)
for k in ['linear','poly','rbf','sigmoid']:
    clf = svm.SVR(kernel=k)
    clf.fit(x_train, y_train)
    confidence = clf.score(x_test, y_test)
    print(k,confidence)

linear 0.9684073874672026
poly 0.7629921653104803
rbf 0.9524870560455925
sigmoid -2.9627473463595755


In [19]:
clf_pred=clf.predict(x_test)

from sklearn.metrics import mean_squared_error

print("MSE:")
mean_squared_error(y_test,clf_pred)

MSE:


167.67457038179506